In [1]:
import alphaspace2 as al
import mdtraj
import numpy as np
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import fcluster, linkage
from alphaspace2.functions import _binCluster, _group
from alphaspace2.Cluster import _DPocket
import pandas as pd
import os

In [2]:
from communities import genCommunityPocket,CoreCluster

In [3]:
import features

### In the first part we calculate the beta clusters

In [4]:
prot = mdtraj.load('protein_noH/AF-model.pdb')
al.annotateVinaAtomTypes(pdbqt='protein_pdbqt_noH/AF-model.pdbqt', receptor=prot)

/home/cyang/miniconda3/lib/python3.8/site-packages/mdtraj/formats/pdb/pdbfile.py:196: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


In [5]:
ss_prot = al.Snapshot()
ss_prot.run(prot)

In [6]:
def Write_betaAtoms(ss, pdb_file):
    betaAtoms = open(pdb_file,'w')
    count = 1
    c = 1
    for p in ss.pockets:
        for betaAtom in p.betas:
            count = count+1
            coord = betaAtom.centroid
            ASpace = '%.1f'%betaAtom.space
            Score = '%.1f'%betaAtom.score
            atomtype = betaAtom.best_probe_type
            x, y, z  = '%.3f'%coord[-3], '%.3f'%coord[-2], '%.3f'%coord[-1]
            line = 'ATOM  ' + str(count).rjust(5) + str(atomtype).upper().rjust(5) + ' BAC' + str(c).rjust(5) + '     ' + str(x).rjust(8) + str(y).rjust(8) + str(z).rjust(8) + ' ' + str(ASpace).rjust(5) + ' ' + str(Score).rjust(5) + '           %s\n'%atomtype
            betaAtoms.write(line)
    betaAtoms.close()

In [7]:
Write_betaAtoms(ss_prot, 'betaAtoms/AF_betaAtoms.pdb')

### For the next cell, we store pocket properties into dictionaries which will be used for generating the surface communities

In [8]:
temp_space_dict = {}
temp_coords_dict = {}
temp_contact_dict = {}
temp_score_dict = {}
temp_centroids_dict = {}
for px,pocket in enumerate(ss_prot.pockets):
    temp_coords_dict[px] = [list(b.xyz) for b in pocket.betas]
    temp_score_dict[px] = np.array([min(b.scores) for b in  pocket.betas])
    temp_space_dict[px] = np.array([b.space for b in  pocket.betas])
    temp_contact_dict[px] = list(pocket.lining_atoms_idx)
    temp_centroids_dict[px] = pocket.centroid

In [9]:
prot_coords = prot.xyz[0]*10 

### Generating communities for the CDK2 surface using the genCommunityPocket function

In [10]:
surface_communities = genCommunityPocket(prot_coords, temp_centroids_dict, temp_space_dict, \
                                            temp_contact_dict, temp_score_dict, \
                                            corecut = 100, auxcut = 30, tight_option = True, tight_cutoff_core = 12.5, tight_cutoff_aux = 6.5)

### Calculate more features of beta clusters community

In [11]:
surface_communities_props = {}
for cx,community in surface_communities.items():
    temp_coords_array = []
    for pock in community['core_pockets'] + community['aux_pockets']:
        temp_coords_array.extend(temp_coords_dict[pock])
    temp_coords_array = np.array(temp_coords_array)
    volume = features._get_grid_volume(temp_coords_array)
    occluded_asa = features._get_pharmacophore_fingerprint(prot,temp_coords_array)
    surface_communities_props[cx] = {}
    surface_communities_props[cx]['space'] = community['space']
    surface_communities_props[cx]['score'] = community['score']
    surface_communities_props[cx]['volume'] = volume
    surface_communities_props[cx]['occluded_asa'] = occluded_asa['Total_OASA']

### We save the pocket communities into pandas dataframe for easier viewing and data manipulation

In [12]:
community_data = pd.DataFrame.from_dict(surface_communities_props,orient='index')
community_data.head(10)

,space,score,volume,occluded_asa
0,1957.0,-27.21,829.875,789.874613
1,1307.0,-15.54,616.500,723.399201
2,1255.0,-18.45,582.125,604.536966
3,970.0,-16.32,659.625,629.201766
4,606.0,-10.86,439.125,405.932265
5,585.0,-9.64,331.000,359.775833
6,503.0,-6.82,255.500,294.027993
7,465.0,-7.80,285.500,302.144754
8,393.0,-7.37,287.500,322.287796
9,337.0,-6.94,115.000,73.242363


In [19]:
pock_list = []

for cx,community in surface_communities.items():
    if cx in [2]:
        for pock in community['core_pockets'] + community['aux_pockets']:
            pock_list.append(pock)
pock_list

[156, 161, 163, 164, 165, 155, 158]

In [20]:
def Write_betaAtoms(ss, pock_list, pdb_file):
    selected_P = []
    for i in pock_list:
        selected_P.append([p for p in ss.pockets if p.index == i][0])
    betaAtoms = open(pdb_file,'w')
    for c,p in zip(pock_list, selected_P):
        count = 0
        for betaAtom in p.betas:
            count = count+1
            coord = betaAtom.centroid
            ASpace = '%.1f'%betaAtom.space
            Score = '%.1f'%betaAtom.score
            atomtype = betaAtom.best_probe_type
            x, y, z  = '%.3f'%coord[-3], '%.3f'%coord[-2], '%.3f'%coord[-1]
            line = 'ATOM  ' + str(c).rjust(5) + '  BAO BAC' + str(count).rjust(5) + '     ' + str(x).rjust(8) + str(y).rjust(8) + str(z).rjust(8) + ' ' + str(ASpace).rjust(5) + ' ' + str(Score).rjust(5) + '           %s\n'%atomtype
            betaAtoms.write(line)
    betaAtoms.close()

In [21]:
Write_betaAtoms(ss_prot, pock_list, '../structure_data/prepare_box/inactive/box2_Helix.pdb')

### In this section, we present some scripts that can be use to visualize CDK2 communities along with the corresponding pictures. We will be saving the pocket communities in the CDK2_Communities/pocket_communities folder

In [13]:
protein_topology = []
for resatom in prot.top.atoms:
    res,atom_index,res_index,atom,element = resatom.residue.name, resatom.index, resatom.residue.index, resatom.name, resatom.element.symbol
    protein_topology.append([res,atom_index,res_index,atom,element])
protein_topology = np.array(protein_topology)

In [14]:
for sx,data in surface_communities.items():
    community_pdb = []
    lining_atoms = set()
    for pock in data['core_pockets'] + data['aux_pockets']:
        score = np.sum(temp_score_dict[pock])
        centroid = temp_centroids_dict[pock]
        lining_atoms.update(list(temp_contact_dict[pock]))
        if score <= -2.5:
            res = 'BHI'
        elif score > -2.5 and score <= -1.5:
            res = 'BMI'
        elif score > -1.5:
            res = 'BLI'
        community_pdb.append('ATOM  '+str(pock).rjust(5)+'  '+'BAO'+' '+res+'   '+str(pock).rjust(3)+'    '+str(round(centroid[0],3)).rjust(8)+str(round(centroid[1],3)).rjust(8)+str(round(centroid[2],3)).rjust(8)+str(0.0).rjust(6)+str(0.00).rjust(6)+'           C')
    community_pdb.append('TER')
    lining_atoms = list(lining_atoms)
    lining_atoms.sort()
    for top,coord in zip(protein_topology[lining_atoms],prot_coords[lining_atoms]):
        res,atom_index,res_index,atom,element = top 
        community_pdb.append('ATOM  '+str(atom_index).rjust(5)+'  '+atom.ljust(3)+' '+res+'   '+str(res_index).rjust(3)+'    '+str(round(coord[0],3)).rjust(8)+str(round(coord[1],3)).rjust(8)+str(round(coord[2],3)).rjust(8)+str(0.0).rjust(6)+str(0.00).rjust(6)+'           '+element)
    with open('pocket_AF/pocket_communities/community_'+str(sx)+'.pdb','w') as f:
        f.write('\n'.join(community_pdb))

In [15]:
with open('pocket_AF/pocket_communities/protein.pdb','w') as f:
    for top,coord in zip(protein_topology,prot_coords):    
        res,atom_index,res_index,atom,element = top 
        f.write('ATOM  '+str(atom_index).rjust(5)+'  '+atom.ljust(3)+' '+res+'   '+str(res_index).rjust(3)+'    '+str(round(coord[0],3)).rjust(8)+str(round(coord[1],3)).rjust(8)+str(round(coord[2],3)).rjust(8)+str(0.0).rjust(6)+str(0.00).rjust(6)+'           '+element+'\n')